
---

# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./output_images/camera_dist.jpg "Camera Undistortion"
[image2]: ./output_images/camera_dist_warp.jpg "Camera Undistortion and Warping"
[image3]: ./output_images/test_img_dist.jpg "Image Undistortion"
[image4]: ./output_images/test_img_grad.jpg "Gradient Thresholding"
[image5]: ./output_images/test_img_hls.jpg "Color Thresholding"
[image6]: ./output_images/test_img_stacked.jpg "Color and Grad Stacked"
[image7]: ./output_images/test_img_warp.jpg "Warped Test Image"
[image8]: ./output_images/test_img_histogram.jpg "Histogram"
[image9]: ./output_images/test_img_unwarp.jpg "Test Image Unwarped"
[image10]:./output_images/test_img_unwarp_curved.jpg "Test Image Unwarped - Curved"
[video1]: ./output_videos/project_video_annotated.mp4 "Video"

### Camera Calibration

The code for this step is contained in the first code cell of the IPython notebook located in "./Advanced Lane Finding.ipynb".

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result *[Source: Udacity Lecture Notes]*: 


![alt text][image1]

The undistorted image is then corrected for perspective using `cv2.getPerspectiveTransform()`. This is done by feeding in coordinates of a rectangular as seen in the unwarped raw source and how the similar rectangle would look in the destination/target perspective. The image below shows both undistortion and warping (perspective correction) done in one step:

![alt text][image2]

### Image Pipeline 

#### 1. Distortion Correction
To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one below. On careful observation, we can see the effect of distortion correction near the edges of the image. This is done using `cv2.undistort()` function. The jupyter notebook containes this code in *Cell 2, Line 6*

![alt text][image3]

#### 2. Thresholding

Initially, I used a combination of color and gradient thresholds to generate a binary image (thresholding steps at lines 18 through 128 in *Cell 2* of the jupyter notebook). In addition to this, I also tried using a trapezoidal mask to reject non-lane data points detected in the image after thresholding (*Cell 2, line 128 onwards*). After a few trials, I ended up using only color thresholding while leaving out gradient thresholding and masking. I observed that the binary image was less noisier with these settings and also didn't really need any masking either. For color thresholding, I found that converting the image to the HLS space was more effective than the usual RGB colorspace. In the HLS, I only used Saturation and Lightness threhsolds which were enough to distinctly identify yellow and white lane lines while ignoring the tree shadows. The images below show these results in the following order: gradient threhsolding, color thresholding, color and gradient thresholding stacked in one image, with gradient in green and color threhsolding in blue. 

![alt text][image4]
![alt text][image5]
![alt text][image6]

#### 3. Perspective Transform

The code for my perspective transform includes a function called `image_warp()` (*Cell 3, Line 6*) in the jupyter notebook `Advanced Lane Finding.ipynb`. The `image_warp()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
src_pts = np.array([[278,675],[552,480],[734,480],[1040,675]])
dst_pts = np.array([[418,720],[418,0],[900,0],[900,720]])
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 278, 675      | 418, 720      | 
| 552, 480      | 418, 0        |
| 734, 480      | 900, 0        |
| 1040,675      | 900, 720      |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image. The following image shows the warping effect. The red rectangle shows the `src` points and the blue rectangle shows the `dst` destination poitns:

![alt text][image7]

#### 4. Lane Line Polynomial Fitting

The lane finding algorithm ( *Cell 4* ) is based on the 'window search' methodology shown in the Udacity lectures. For the video implementation, the idea is that once a lane is identified, we no longer do a blind window seach but search in the vicinity of the lanes identified in the previous frame and only after a failure in detecting a line in that region, we re-do the window search. Coming back to window search, the starting point of the search is identified using a histogram plot of the non-zero data points in the binary image. Two peaks, one in left half and right half each, represent the centre of left and right lane roots respectively. Starting here, a window search is done going from bottom to top in the image, tracking through the 'dense' region. The image below shows the histogram equivalent of the warped binary image.

![alt text][image8]

Having said that, once the left and right lane indices are indentified, a second order polynomial fit is obtained using these points, which is used to represent left and right lane lines as shown in the subsequent sections.

#### 5. Radius of Curvature

*Cell 6* in the jupyter notebook `Advanced Lane Finding.ipynd` shows the radius of curvature calcualtion. The key is to first carefully measure the lane width in pixels in the warped image and then equate that pixel count to the lane width in meters, which is assumed as a standard 3.7 m here. Similarly, the length of the road patch used for perspective transform is assumed to be 30 m and the corresponding pixel count is used to calibrate the pixel/m conversion. The standard radius of curvature expression for a second degree polynomial is then used to caluclate the radius. For video implementation, the curvature of the lane with more data points is used as the lane curvature as that lane line is more 'reliable'.

##### Offset
The vehicle position with respect to the lane center is also calculated. The idea is to use the image center and the mid point of left and right lanes and use these points to identify whether the vehicle is right or left of the lane center with the fact that the camera is mounted right at the middle position of the car front.

#### 6. Unwarping and projecting the detected lane back on the road

I implemented this step in cell 7 of the jupyter notebook. Used the `Minv`, the unwarping matrix found using the same warping function as used before, but the source and destination swapped. The result of this unwarping and projecting the lane region back on the road is shown in the two images below, one of straight line patch, other of a curved patch of the road. The images on the left also show the 'lane vicinity' or 'lane detection region' where the subsequent lane lines are searched for one the lane lines are detected using the window search method:

![alt text][image9]
![alt text][image10]

---

### Pipeline (video)

Some additional logic (* Cell 8 and 9 *) has been added in the video pipeline to improve the robustness of the lane detection process. It tracks the lane history and uses it in case of lane detection failure in the current frame. It also implements fisrt order filtering on the curve fits to smoothen the lane curves frame by frame. Furthermore, once the window search has identified the lane lines, it turns to the other part of the algorithm which looks for lane lines only in the vicinity of the lines detected in the last frame and on failure to do so, it goes back to window search method. Lastly, in order to do a sanity check on the detected lines, it looks for average lane width and its standard deviation (* using the function `meanLaneWidth()` *) throughout the image and if things are out of bounds, it rejects that detection and falls back to the history. 

Here's a [link to my video result][video1]

### Discussion

Here are a few points I want to discuss about the project. 
While implementing the video pipeline, I had to make sure that proper failsafes are in place in case of undetected or partially/incorrectly detected lane lines in a particular frame. Another important addition was the logic which does a sanity check on the quality of lane detection by looking at the average lane width and its standard deviation across the image. There are still some challenges when this logic is run on advanced difficulty videos, where it fails to give desirable results. The algorithm fails in catching frequent sharp turns with rapidly changing light conditions. I think improving the thresholding, maybe by supplementing the HSL with gradient with proper threhsolding or RGB criteria and also adding some sanity check on the lane curvature and parallelism can be the next things that can be done here to improve the detection logic.
Furthermore, this algorithm may not work well in rain and nigh time driving scenarios because the thresholds tuned here would not be appropriate for those scenarios. An intelligent, adaptive threshold tuning routine based on the weather and lighting conditions would be a way to ensure this works under varying conditions. Lastly, this may also not work in case of other vehicles coming closer, changing lanes/ hiding the lane lines. An algorithm which detects/ identifies the road traffic would be a suitable addendum here.
